In [2]:
# Here is an example of what we are trying to acheive:
# %%html
# <video controls autoplay><source src="https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4" type="video/mp4"></video>

SyntaxError: invalid syntax (628573827.py, line 3)

Install dependencies

In [1]:
# ! pip install setuptools==65.5.0

  Using cached setuptools-65.5.0-py3-none-any.whl (1.2 MB)


In [ ]:
# Already installed 
%%capture
# !apt install python-opengl
# !apt install ffmpeg
# !apt install xvfb
# !apt install swig cmake

In [3]:
# install virtual display dependencies
# !pip install pyglet==1.5
# !pip3 install pyvirtualdisplay

In [2]:
# virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [5]:
# install project dependencies
!pip install gym==0.22
!pip install imageio-ffmpeg
!pip install huggingface_hub
!pip install gym[box2d]==0.22

For the core implementation of PPO we're going to use the excellent [Costa Huang](https://costa.sh/) tutorial.
- In addition to the tutorial, to go deeper you can read the 37 core implementation details: https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/

👉 The video tutorial: https://youtu.be/MEt6rrxH8W4

In [2]:
from IPython.display import HTML

# HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/MEt6rrxH8W4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Add Hugging Face integration

Add dependencies we need to push our model to the hub

In [20]:
# Install Weights and Biases package as that is an optional dependency passed in as a flag to the training script through the main function
!pip install wandb

  Using cached wandb-0.15.5-py3-none-any.whl (2.1 MB)
  Using cached click-8.1.4-py3-none-any.whl (98 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached sentry_sdk-1.27.1-py2.py3-none-any.whl (211 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
  Using cached protobuf-4.23.4-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=be176beb22b9e07c3d16a689ce691462799e9a2a9d3523c162f5f657a139a465
  Stored in directory: /home/raj/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [19]:
# Install the latest version of setuptools to avoid the error above installing wandb
! pip install --upgrade setuptools

  Using cached setuptools-68.0.0-py3-none-any.whl (804 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0


In [21]:
# define make_env function needed to setup the environment in the main function
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.Monitor(env, f"videos/{run_name}")
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk
    

In [23]:
# define the Agent class that will be called from the main function
import torch.nn as nn
import torch

class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01),
        )

    def get_value(self, x):
        return self.critic(x)
    
    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)

In [25]:
# install tensorboard
!pip install tensorboard

  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached grpcio-1.56.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Using cached google_auth-2.21.0-py2.py3-none-any.whl (182 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached tensorboard_data_server-0.7.1-py3-none-manylinux2014_x86_64.whl (6.6 MB)
  Using cached Werkzeug-2.3.6-py3-none-any.whl (242 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Attempting uninstall: urllib3
    Foun

Add new argument in parse_args() function to define the repo-id where we want to push the model

In [12]:
! pip3 install setuptools==40.8.0

  Using cached setuptools-40.8.0-py2.py3-none-any.whl (575 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0


In [34]:
! pip3 install torch torchvision torchaudio


In [35]:
# setup cuda device
# check if gpu is available
"""check if gpu is available"""
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [39]:
# define _evaluate_agent function for Step 3 of the package_to_hub function
import numpy as np
def _evaluate_agent(env, n_eval_episodes, policy):
    """ Evaluate the agent for ``n_eval_episodes`` episodes and returns the mean and std of the rewards.
    :param env: The evaluation environment
    :param n_eval_episodes: The number of episodes to evaluate the agent
    :param policy: The policy to use to act by the agent
    """

    episode_rewards = []
    for episode in range(n_eval_episodes):
        state = env.reset()
        step = 0
        done = False
        total_rewards_ep = 0

        while done is False:
            state = torch.Tensor(state).to(device)
            action, _, _, _ = policy.get_action_and_value(state)
            new_state, reward, done, info = env.step(action.cpu().numpy())
            total_rewards_ep += reward
            if done:
                break
            state = new_state
            episode_rewards.append(total_rewards_ep)
        mean_reward = np.mean(episode_rewards)
        std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

In [40]:
# define record_video() function for Step 4 of the package_to_hub function
import torch
import imageio_ffmpeg
def record_video(env, policy, out_directory, fps=30):
    images = []
    done = False
    state = env.reset()
    img = env.render(mode="rgb_array")
    images.append(img)
    while not done:
        state = torch.Tensor(state).to(device)
        # take the action (index) that have the maximum expected future reward given that state
        action, _, _, _ = policy.get_action_and_value(state)
        state, reward, done, info = env.step(action.cpu().numpy()) # We directly put next_state = state for recording logic
        img = env.render(mode="rgb_array")
        images.append(img)
    imageio_ffmpeg.imwrite(os.path.join(out_directory, "preview.jpg"), [np.array(img) for i, img in enumerate(images)], fps=fps)


In [41]:
# Function _generate_model_card definition for Step 5 of the package_to_hub function

# First define the dependent generate metadata function which is needed for the _generate_model_card
def generate_metadata(model_name, env_id, mean_reward, std_reward):
  """
  Generate the metadata for the model card
  :param model_name: name of the model
  :env_id: name of the environment
  :mean_reward: mean reward of the agent
  :std_reward: standard deviation of the mean reward of the agent
  """
  metadata = {}
  metadata["tags"] = [
    env_id,
    "ppo",
    "deep-reinforcement-learning",
    "reinforcement-learning",
    "custom-implementation",
    "deep-rl-course"
  ]

  # Add metrics
  eval = metadata_eval_result(
    model_pretty_name=model_name,
    task_pretty_name="reinfrocement-learning",
    task_id="reinforcement-learning",
    metrics_pretty_name="mean_reward",
    metrics_id="mean_reward",
    metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
    dataset_pretty_name=env_id,
    dataset_id=env_id,
  )

  # Merges both dictionaries
  metadata = {**metadata, **eval}

  return metadata
      

# generate model card
def _generate_model_card(model_name, env_id, mean_reward, std_reward, hyperparameters):
  """
  Generate the model card for the Hub
  :param model_name: name of the model
  :env_id: name of the environment
  :mean_reward: mean reward of the agent
  :std_reward: standard deviation of the mean reward of the agent
  :hyperparameters: training arguments
  """
  # Step 1: Select the tags
  metadata = generate_metadata(model_name, env_id, mean_reward, std_reward)

  # Transform the hyperparameters namespace into a string
  converted_dict = vars(hyperparameters)
  converted_str = str(converted_dict)
  converted_str = converted_str.split(", ")
  converted_str = "\n".join(converted_str)

  # Step 2: Generate the model card
  model_card = f"""
  # PPO Agent Playing {env_id}

  This is a trained model of a PPO agent playing {env_id}.

  # Hyperparameters
  ```python
  {converted_str}
  ```
  """
  return model_card, metadata

In [31]:
# understanding nested triple quotes
str = "Hello world!"
card = f""" 
# Demo:
```python
{str}
```
"""
print(card)

 
# Demo:
```python
Hello world!
```



In [9]:
# Function save_model_card needed for step 5 of the package_to_hub function
def _save_model_card(local_path, generated_model_card, metadata):
    """
    Saves a model card to the repository.
    :param local_path: repository directory
    :param generated_model_card: generated model card by _generate_model_card()
    :param metadata: generated metadata
    """

    readme_path = local_path / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf-8") as f:
            readme = f.read()
    else:
        readme = generated_model_card

    with readme_path.open("w", encoding="utf-8") as f:
        f.write(readme)

    # Save the metrics to Readme metadata
    metadata_save(readme_path, metadata)


In [10]:
# Function _add_logdir needed for step 6 of the package_to_hub function
def _add_logdir(local_path: Path, logdir: Path):
    """
    Adds the logdir to the repository.
    :param local_path: repository directory
    :param logdir: logdir to add
    """
    if logdir.exists() and logdir.is_dir():
        # Add the logdir to the repository under new dir called logs
        repo_logdir = local_path / "logs"

        # Delete current logs if they exist
        if repo_logdir.exists():
            shutil.rmtree(repo_logdir)

        # Copy logdir into repo logdir
        shutil.copytree(logdir, repo_logdir)
        

In [36]:
import torch
from wasabi import msg
from pathlib import Path
from huggingface_hub import HfApi, HfFolder, Repository, upload_folder
from typing import Optional
import tempfile
import json
import datetime

def package_to_hub(repo_id,
                   model,
                   hyperparameters,
                   eval_env,
                   video_fps=30,
                   commit_message="Push agent to the hub",
                   token=None,
                   logs=None
                   ):
    """
    Evaluate, Generate a video and upload a model to Hugging Face Hub.
    This method does the complete pipeline:
    - It evaluates the model
    - It generates the model card
    - It generates a replay video of the agent
    - It pushes everything to the hub
    :param repo_id: id of the model repository from the Hugging Face Hub
    :param model: trained model
    :param eval_env: environment used to evaluate the agent
    :param fps: number of fps for rendering the video
    :param commit_message: commit message
    :param logs: directory on local machine of tensorboard logs you'd like to upload   
    """

    msg.info(
        "This function will save, evaluate, generate a video of your agent, "
        "create a model card and push everything to the hub. "
        "It might take up to 1min. \n "
        "This is a work in progress: if you encounter a bug, please open an issue."
    )

    # Step 1: Clone or create the repo
    repo_url = HfApi().create_repo(
        repo_id=repo_id,
        token=token,
        private=False,
        exist_ok=True,
    )

    with tempfile.TemporaryDirectory() as tmpdirname:
        tempdirname = Path(tmpdirname)

        # Step 2: Save the model
        torch.save(model.state_dict(), tempdirname / "model.pt")

        # Step 3: Evaluate the model and build JSON
        mean_reward, std_reward = _evaluate_agent(eval_env,
                                                  10,
                                                  model)
        # First get datetime
        eval_datetime = datetime.datetime.now()
        # Then convert to string
        eval_form_datetime = eval_datetime.isoformat()

        # Build JSON
        evaluate_data = {
            "env_id": hyperparameters.env_id,
            "mean_reward": mean_reward,
            "std_reward": std_reward,
            "n_eval_episodes": 10,
            "eval_datetime": eval_form_datetime,
        }

        # Write a JSON file
        with open(tmpdirname / "results.json", "w") as f:
            json.dump(evaluate_data, f)

        # Step 4: Generate a video
        video_path = tmpdirname / "replay.mp4"
        record_video(eval_env, model, video_path, video_fps=video_fps)

        # Step 5: Generate a model card
        generated_model_card, metadata = _generate_model_card("PPO",
                                                                hyperparameters.env_id,
                                                                mean_reward,
                                                                std_reward,
                                                                hyperparameters)
        _save_model_card(tmpdirname, generated_model_card, metadata)

        # Step 6: Add logs if needed
        if logs:
            _add_logdir(tmpdirname, Path(logs))

        msg.info(f"Pushing the model to the hub under the id {repo_id}...")

        repo_url = upload_folder(
            repo_id=repo_id,
            folder_path=tmpdirname,
            path_in_repo="",
            commit_message=commit_message,
            token=token,
        )
        
        msg.info(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")
        return repo_url


In [48]:
# Adding HuggingFace argument
import argparse
import os
from distutils.util import strtobool

def parse_args():
    # fmt: off
    parser = argparse.ArgumentParser()
    parser.add_argument("--exp-name", type=str, default=os.path.abspath('').rstrip(".py"), 
                        help="Name of the experiment")
    parser.add_argument("--seed", type=int, default=42, help="Seed for reproducibility")
    parser.add_argument("--torch-deterministic", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True, help="If Toggled, `torch.backends.cudnn.deterministic=False`")
    parser.add_argument("--cuda", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, cuda will be enabled by default")
    parser.add_argument("--track", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
        help="if toggled, this experiment will be tracked with Weights and Biases")
    parser.add_argument("--wandb-project-name", type=str, default="cleanRL",
        help="the wandb's project name")
    parser.add_argument("--wandb-entity", type=str, default=None,
        help="the entity (team) of wandb's project")
    parser.add_argument("--capture-video", type=lambda x: bool(strtobool(x)), default=False, nargs="?", const=True,
        help="weather to capture videos of the agent performances (check out `videos` folder)")
    
    # Alogrithm specific arguments
    parser.add_argument("--env-id", type=str, default="LunarLander-v2",
        help="the id of the environment")
    parser.add_argument("--total-timesteps", type=int, default=20000, # should set it to 1_000_000 when running on your own
        help="total timesteps of the experiments")
    parser.add_argument("--learning-rate", type=float, default=2.5e-4,
        help="the learning rate of the optimizer")
    parser.add_argument("--num-envs", type=int, default=16,
        help="the number of parallel game environments")
    parser.add_argument("--num-steps", type=int, default=1024,
        help="the number of steps to run in each environment per policy rollout")
    parser.add_argument("--anneal-lr", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggle learning rate annealing for policy and value networks")
    parser.add_argument("--gae", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Use GAE for advantage computation")
    parser.add_argument("--gamma", type=float, default=0.999,
        help="the discount factor gamma")
    parser.add_argument("--gae-lambda", type=float, default=0.98,
        help="the lambda for the general advantage estimation")
    parser.add_argument("--num-minibatches", type=int, default=64,
        help="the number of mini-batches")
    parser.add_argument("--update-epochs", type=int, default=4,
        help="the K epochs to update the policy")
    parser.add_argument("--norm-adv", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles advantages normalization")
    parser.add_argument("--clip-coef", type=float, default=0.2,
        help="the surrogate clipping coefficient")
    parser.add_argument("--clip-vloss", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles whether or not to use a clipped loss for the value function, as per the paper.")
    parser.add_argument("--ent-coef", type=float, default=0.01,
        help="coefficient of the entropy")
    parser.add_argument("--vf-coef", type=float, default=0.5,
        help="coefficient of the value function")
    parser.add_argument("--max-grad-norm", type=float, default=0.5,
        help="the maximum norm for the gradient clipping")
    parser.add_argument("--target-kl", type=float, default=None,
        help="the target KL divergence threshold")

    # Adding HuggingFace argument    

    parser.add_argument(
        "--repo-id",
        type=str,
        default="RajkNakka/ppo-LunarLander-v2-unit-8",
        help="id of the model repository from the Hugging Face Hbu {username/repo-name}",
    )

    args = parser.parse_args()
    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    # fmt: on
    return args

In [50]:
# The main function
# Imports needed for functions defined below

from huggingface_hub import HfApi, upload_folder
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import tempfile
import json
import shutil
import imageio_ffmpeg
import random
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions.categorical as Categorical
from torch.utils.tensorboard import SummaryWriter

from wasabi import Printer

msg = Printer()

if __name__ == "__main__":
    
    # if running from command line use this
    # args = parse_args()
    
    # to run in jupyter notebook use this
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--exp-name", type=str, default=os.path.abspath('').rstrip(".py"))
    # parser.add_argument("--seed", type=int, default=42, help="Seed for reproducibility")

    args = parse_args()

    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
    if args.track:
        import wandb

        wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
    writer = SummaryWriter(log_dir=f"runs/{run_name}")
    writer.add_text(
        "hyperparams", 
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{k}|{v}|" for k, v in vars(args).items()])),
    )

    # Try not to modify: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # env setup
    envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, i, args.seed +i, i, args.capture_video, run_name) for i in range(args.num_envs)]
    )
    assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

    agent = Agent(envs).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)

    # ALGO Logic: Storage setup
    obs = torch.zeros((args.num_steps, args.num_envs) + envs.single_observation_space.shape).to(device)
    actions = torch.zeros((args.num_steps, args.num_envs) + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((args.num_steps, args.num_envs)).to(device)
    rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
    dones = torch.zeros((args.num_steps, args.num_envs)).to(device)
    values = torch.zeros((args.num_steps, args.num_envs)).to(device)

    # TRY NOT TO MODIFY: start the game
    global_step = 0
    start_time = time.time()
    # Note how `next_obs` and `next_done` are used; their usage is equivalent to `obs[1:]` and `dones[1:]`
    next_obs = torch.Tensor(envs.reset()).to(device)
    next_done = torch.zeros(args.num_envs).to(device)
    num_updates = args.total_timesteps // args.batch_size

    for update in range(1, num_updates+1):
        # Annealing the rate if instructed to do so.
        if args.anneal_lr:
            frac = 1.0 - (update - 1.0) / num_updates
            lrnow = args.learning_rate * frac
            optimizer.param_groups[0]["lr"] = lrnow

        for step in range(0, args.num_steps):
            global_step += 1 * args.num_envs
            obs[step] = next_obs
            dones[step] = next_done

            # ALGO LOGIC: put action logic here
            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(next_obs)
                values[step] = value.flatten()
            actions[step] = action
            logprobs[step] = logprob

            # TRY NOT TO MODIFY: execute the game and log data.
            next_obs, reward, done, info = envs.step(action.cpu().numpy())
            rewards[step] = torch.Tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(done).to(device)

            for item in info:
                if "episode" in item.keys():
                    print(f"global_step={global_step}, episode_reward={item['episode']['r']}")
                    writer.add_scalar("charts/episode_reward", item['episode']['r'], global_step)
                    writer.add_scalar("charts/episode_length", item['episode']['l'], global_step)
                    break

        # bootstrap value if not done
        with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            if args.gae:
                advantages = torch.zeros_like(rewards).to(device)
                lastgaelam = 0
                for t in reversed(range(args.num_steps)):
                    if t == args.num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        nextvalues = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t+1]
                        nextvalues = values[t+1]
                    delta = rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                    advantages[t] = lastgaelam = delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                returns = advantages + values
            else:
                returns = torch.zeros_like(rewards).to(device)
                for t in reversed(range(args.num_steps)):
                    if t == args.num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        next_return = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t+1]
                        next_return = returns[t+1]
                    returns[t] = rewards[t] + args.gamma * nextnonterminal * next_return
                advantages = returns - values

        # flatten the batch
        b_obs = obs.reshape((-1,)+envs.single_observation_space.shape)
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,)+envs.single_action_space.shape)
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        # Optimizaing the policy and value network
        b_inds = np.arange(args.batch_size)
        clipfracs = []
        for i_epoch_pi in range(args.update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, args.batch_size, args.minibatch_size):
                end = start + args.minibatch_size
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalues = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl divergence using logratio
                    old_approx_kl = (-logratio).mean()
                    approxkl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > args.clip_coef).float().mean().item()]

                # calculate advantages
                mb_advantages = b_advantages[mb_inds]
                if args.norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                # calculate policy loss
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1.0 - args.clip_coef, 1.0 + args.clip_coef)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # calculate value loss using either the clipped value or MSE
                newvalue = newvalue.view(-1)
                if args.clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(newvalues - b_values[mb_inds], -args.clip_coef, args.clip_coef)
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                # calculate entropy loss
                entropy_loss = entropy.mean()
                loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
                optimizer.step()

                if args.target_kl is not None and approxkl > args.target_kl:
                    break

            y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
            var_y = np.var(y_true)
            explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

            # TRY NOT TO MODIFY: record rewards for plotting purposes
            writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
            writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
            writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
            writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
            writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
            writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
            writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
            writer.add_scalar("losses/explained_variance", explained_var, global_step)
            print("SPS:", int(global_step / (time.time() - start_time)))
            writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

        envs.close()
        writer.close()

        # Create the evaluation environment
        # eval_env = gym.make(args.env_id)

        # package_to_hub(repo_id = args.repo_id,
        #             model = agent, # The model we want to save
        #             hyperparameters = args,
        #             eval_env = gym.make(args.env_id),
        #             logs= f"runs/{run_name}",
        #             )



usage: ipykernel_launcher.py [-h] [--exp-name EXP_NAME] [--seed SEED]
                             [--torch-deterministic [TORCH_DETERMINISTIC]]
                             [--cuda [CUDA]] [--track [TRACK]]
                             [--wandb-project-name WANDB_PROJECT_NAME]
                             [--wandb-entity WANDB_ENTITY]
                             [--capture-video [CAPTURE_VIDEO]]
                             [--env-id ENV_ID]
                             [--total-timesteps TOTAL_TIMESTEPS]
                             [--learning-rate LEARNING_RATE]
                             [--num-envs NUM_ENVS] [--num-steps NUM_STEPS]
                             [--anneal-lr [ANNEAL_LR]] [--gae [GAE]]
                             [--gamma GAMMA] [--gae-lambda GAE_LAMBDA]
                             [--num-minibatches NUM_MINIBATCHES]
                             [--update-epochs UPDATE_EPOCHS]
                             [--norm-adv [NORM_ADV]] [--clip-coef CLIP_COEF]
        

SystemExit: 2

/home/raj/repos/HF-DeepRL/8-Proximal-Policy-Optimization/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [51]:
# login to huggingface from notebook
from huggingface_hub import notebook_login
notebook_login()


In [53]:
# Train the model without pushing to the hub: make sure the package_to_hub function is commented out
!python ppo.py 


global_step=1008, episodic_return=-111.396484375
global_step=1024, episodic_return=-249.04827880859375
global_step=1104, episodic_return=-164.42489624023438
global_step=1280, episodic_return=-356.3940734863281
global_step=1392, episodic_return=-260.03045654296875
global_step=1408, episodic_return=-130.0479736328125
global_step=1488, episodic_return=-78.94027709960938
global_step=1536, episodic_return=-237.42181396484375
global_step=1552, episodic_return=-209.61349487304688
global_step=1600, episodic_return=-208.95553588867188
global_step=1760, episodic_return=-364.6192321777344
global_step=1904, episodic_return=-113.28790283203125
global_step=2016, episodic_return=-107.48539733886719
global_step=2064, episodic_return=-143.09176635742188
global_step=2288, episodic_return=-77.35099029541016
global_step=2352, episodic_return=-134.1401824951172
global_step=2448, episodic_return=-193.4730224609375
global_step=2512, episodic_return=-86.79818725585938
global_step=2672, episodic_return=-79.32

In [4]:
# Train, evaluate and push to the hub
!python ppo.py --total-timesteps 100 --capture-video False


SPS: 90
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast
X Error of failed request:  BadValue (integer parameter out of range for operation)
  Major opcode of failed request:  150 (GLX)
  Minor opcode of failed request:  3 (X_GLXCreateContext)
  Value in failed request:  0x0
  Serial number of failed request:  92
  Current serial number in output stream:  93


In [46]:
# Push the model to the hub
package_to_hub(repo_id = args.repo_id = "RajkNakka/ppo-LunarLander-v2-unit-8",
            model = agent, # The model we want to save
            hyperparameters = args,
            eval_env = gym.make(args.env_id),
            logs= f"runs/{run_name}",
            )

SyntaxError: invalid syntax (2847039533.py, line 2)